In [27]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [28]:
# Load the gun violence dataset
df = pd.read_csv('gun_violence_cleaned.csv')

# Display basic information
print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
pd.set_option('display.max_columns', None)
df.head(20)

Dataset shape: (239677, 43)

First few rows:


,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,num_victims,num_suspects,num_females,num_males,num_adults,num_teens,num_adult_suspects,num_adult_victims,num_teen_suspects,num_teen_victims,num_male_suspects,num_male_victims,num_female_suspects,num_female_victims
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN,4,1,1,1,5,0,1,4,0,0,0,3,1,0
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0,4,1,0,0,4,0,0,4,0,0,0,1,0,0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,0::Unknown||1::Unknown,0::Unknown||1::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,Cotton Club,-82.1377,2.0,NaN,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0,3,2,0,0,5,0,2,3,0,0,2,3,0,0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Off...",39.6518,NaN,-104.8020,NaN,NaN,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0,3,1,1,1,4,0,1,3,0,0,1,2,0,1
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,0::Unknown||1::Unknown,0::Handgun||1::Handgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0,3,1,2,2,3,1,1,2,0,1,0,2,1,1
5,478948,2013-01-07,Oklahoma,Tulsa,6000 block of South Owasso,4,0,http://www.gunviolencearchive.org/incident/47894

In [29]:
# Create a function to count stolen guns
def count_stolen_guns(gun_stolen_str):
    if pd.isna(gun_stolen_str):
        return 0
    # Split by || to get individual guns
    guns = str(gun_stolen_str).split('||')
    # Count how many contain "Stolen" (not "Not-stolen")
    stolen_count = 0
    for gun in guns:
        if '::' in gun:
            status = gun.split('::')[1].strip()
            if status.lower() == 'stolen':
                stolen_count += 1
    return stolen_count

In [30]:
df[["participant_gender", "num_males", "num_females"]].head(40)

,participant_gender,num_males,num_females
0,0::Male||1::Male||3::Male||4::Female,1,1
1,0::Male,0,0
2,0::Male||1::Male||2::Male||3::Male||4::Male,0,0
3,0::Female||1::Male||2::Male||3::Male,1,1
4,0::Female||1::Male||2::Male||3::Female,2,2
5,0::Female||1::Female||2::Female||3::Female||4:...,4,4
6,0::Male||1::Female||2::Male||3::Female||4::Fem...,3,3
7,0::Male||1::Male||2::Male||3::Male||4::Male,0,0
8,0::Male||1::Male||2::Male||3::Male||4::Male,0,0
9,0::Male,0,0


In [31]:
# Fix num_males and num_females using participant_gender

# Store old values for comparison
old_num_males = df['num_males'].copy()
old_num_females = df['num_females'].copy()

def count_gender(participant_gender_str, target_gender):
    """
    Count occurrences of a specific gender in participant_gender string.
    
    Parameters:
    - participant_gender_str: The participant_gender field from the dataset
    - target_gender: "Male" or "Female"
    
    Returns:
    - Count of the specified gender
    """
    if pd.isna(participant_gender_str):
        return 0
    # Split by || to get individual participants
    participants = str(participant_gender_str).split('||')
    # Count how many match the target gender
    count = 0
    for participant in participants:
        if '::' in participant:
            gender = participant.split('::')[1].strip()
            if gender == target_gender:
                count += 1
    return count

# Replace num_males and num_females with corrected counts
df['num_males'] = df['participant_gender'].apply(lambda x: count_gender(x, 'Male'))
df['num_females'] = df['participant_gender'].apply(lambda x: count_gender(x, 'Female'))

# Display comparison
print("="*70)
print("GENDER COUNT CORRECTION")
print("="*70)

print("\nBefore correction:")
print(f"Total males (old): {old_num_males.sum():,}")
print(f"Total females (old): {old_num_females.sum():,}")

print("\nAfter correction:")
print(f"Total males (new): {df['num_males'].sum():,}")
print(f"Total females (new): {df['num_females'].sum():,}")

GENDER COUNT CORRECTION

Before correction:
Total males (old): 43,172
Total females (old): 43,172

After correction:
Total males (new): 304,102
Total females (new): 42,376


In [51]:
df[["participant_gender", "num_males", "num_females"]].head(40)

,participant_gender,num_males,num_females
0,0::Male||1::Male||3::Male||4::Female,3,1
1,0::Male,1,0
2,0::Male||1::Male||2::Male||3::Male||4::Male,5,0
3,0::Female||1::Male||2::Male||3::Male,3,1
4,0::Female||1::Male||2::Male||3::Female,2,2
5,0::Female||1::Female||2::Female||3::Female||4:...,2,4
6,0::Male||1::Female||2::Male||3::Female||4::Fem...,3,3
7,0::Male||1::Male||2::Male||3::Male||4::Male,5,0
8,0::Male||1::Male||2::Male||3::Male||4::Male,5,0
9,0::Male,1,0


In [32]:
# Create a function to count stolen guns
def count_stolen_guns(gun_stolen_str):
    if pd.isna(gun_stolen_str):
        return 0
    # Split by || to get individual guns
    guns = str(gun_stolen_str).split('||')
    # Count how many contain "Stolen" (not "Not-stolen")
    stolen_count = 0
    for gun in guns:
        if '::' in gun:
            status = gun.split('::')[1].strip()
            if status.lower() == 'stolen':
                stolen_count += 1
    return stolen_count

In [33]:
# Create a function to count unknown guns
def count_unknown_guns(gun_stolen_str):
    if pd.isna(gun_stolen_str):
        return 0
    # Split by || to get individual guns
    guns = str(gun_stolen_str).split('||')
    # Count how many contain "Unknown"
    unknown_count = 0
    for gun in guns:
        if '::' in gun:
            status = gun.split('::')[1].strip()
            if status.lower() == 'unknown':
                unknown_count += 1
        elif gun.strip().lower() == '0:unknown':  # Handle the malformed entry
            unknown_count += 1
    return unknown_count

In [34]:
# Create a function to count not-stolen guns
def count_not_stolen_guns(gun_stolen_str):
    if pd.isna(gun_stolen_str):
        return 0
    # Split by || to get individual guns
    guns = str(gun_stolen_str).split('||')
    # Count how many contain "Not-stolen"
    not_stolen_count = 0
    for gun in guns:
        if '::' in gun:
            status = gun.split('::')[1].strip()
            if status.lower() == 'not-stolen':
                not_stolen_count += 1
    return not_stolen_count

In [35]:
# Sum all guns involved
total_guns = df['n_guns_involved'].sum()
print(f"Total guns involved across all incidents: {total_guns:,.0f}\n")

# Apply the function to create new column
df['n_guns_stolen'] = df['gun_stolen'].apply(count_stolen_guns)

# Display results
print(f"Total guns stolen: {df['n_guns_stolen'].sum():,.0f}")
print(f"Incidents with stolen guns: {(df['n_guns_stolen'] > 0).sum():,.0f}\n")

# Apply the function to create new column
df['n_guns_unknown'] = df['gun_stolen'].apply(count_unknown_guns)

# Display results
print(f"Total guns with unknown stolen status: {df['n_guns_unknown'].sum():,.0f}")
print(f"Incidents with unknown gun status: {(df['n_guns_unknown'] > 0).sum():,.0f}\n")

# Apply the function to create new column
df['n_guns_not_stolen'] = df['gun_stolen'].apply(count_not_stolen_guns)

# Display results
print(f"Total guns not stolen: {df['n_guns_not_stolen'].sum():,.0f}")
print(f"Incidents with not-stolen guns: {(df['n_guns_not_stolen'] > 0).sum():,.0f}")

Total guns involved across all incidents: 192,452

Total guns stolen: 17,610
Incidents with stolen guns: 7,350

Total guns with unknown stolen status: 172,861
Incidents with unknown gun status: 132,435

Total guns not stolen: 1,804
Incidents with not-stolen guns: 1,560


In [36]:
# Check date range
print(f"Date range: {df['date'].min()} to {df['date'].max()}")

Date range: 2013-01-01 to 2018-03-31


In [37]:
# Check unique states
print(f"Number of unique states: {df['state'].nunique()}")
print(f"\nAll states in dataset:")
print(sorted(df['state'].unique()))

Number of unique states: 51

All states in dataset:
['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']


In [38]:
# Display gender statistics
print("Gender Statistics Across All Incidents:")
print(f"\nTotal males: {df['num_males'].sum():,.0f}")
print(f"Total females: {df['num_females'].sum():,.0f}")
print(f"Total teens: {df['num_teens'].sum():,.0f}")
print(f"\nTotal male victims: {df['num_male_victims'].sum():,.0f}")
print(f"Total female victims: {df['num_female_victims'].sum():,.0f}")
print(f"Total teen victims: {df['num_teen_victims'].sum():,.0f}")
print(f"\nTotal male suspects: {df['num_male_suspects'].sum():,.0f}")
print(f"Total female suspects: {df['num_female_suspects'].sum():,.0f}")
print(f"Total teen suspects: {df['num_teen_suspects'].sum():,.0f}")

Gender Statistics Across All Incidents:

Total males: 304,102
Total females: 42,376
Total teens: 25,615

Total male victims: 136,395
Total female victims: 30,630
Total teen victims: 12,343

Total male suspects: 167,708
Total female suspects: 11,746
Total teen suspects: 12,850


In [39]:
# Create a heat map of gun violence incidents by state

# Count incidents by state
state_counts = df['state'].value_counts().reset_index()
state_counts.columns = ['state', 'count']

# Map full state names to abbreviations
state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO',
    'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH',
    'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY',
    'District of Columbia': 'DC'
}

state_counts['state_code'] = state_counts['state'].map(state_abbrev)

print(f"Top 10 states by incident count:")
print(state_counts.head(10))
print(f"\nTotal incidents: {state_counts['count'].sum():,}")

# Create choropleth map
fig = go.Figure(data=go.Choropleth(
    locations=state_counts['state_code'],
    z=state_counts['count'],
    locationmode='USA-states',
    colorscale='Reds',
    colorbar_title="Incidents",
    text=state_counts['state'],
    hovertemplate='<b>%{text}</b><br>Incidents: %{z:,}<extra></extra>'
))

fig.update_layout(
    title_text='Gun Violence Incidents by State (2013-2018)',
    geo_scope='usa',
    height=600,
    width=1000
)

fig.show()

Top 10 states by incident count:
            state  count state_code
0        Illinois  17556         IL
1      California  16306         CA
2         Florida  15029         FL
3           Texas  13577         TX
4            Ohio  10244         OH
5        New York   9712         NY
6    Pennsylvania   8929         PA
7         Georgia   8925         GA
8  North Carolina   8739         NC
9       Louisiana   8103         LA

Total incidents: 239,677


In [40]:
# Create a function to determine season from date
def get_season(date_str):
    month = pd.to_datetime(date_str).month
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:  # month in [9, 10, 11]
        return 'Fall'

# Apply the function to create season column
df['season'] = df['date'].apply(get_season)

# Count incidents by season
season_counts = df['season'].value_counts().reindex(['Winter', 'Spring', 'Summer', 'Fall'])

# Create bar chart
fig = go.Figure(data=[
    go.Bar(
        x=season_counts.index,
        y=season_counts.values,
        marker_color=['lightblue', 'lightgreen', 'orange', 'brown'],
        text=season_counts.values,
        textposition='auto',
        texttemplate='%{text:,}',
        hovertemplate='<b>%{x}</b><br>Incidents: %{y:,}<extra></extra>'
    )
])

fig.update_layout(
    title='Gun Violence Incidents by Season (2013-2018)',
    xaxis_title='Season',
    yaxis_title='Number of Incidents',
    height=500,
    width=800,
    showlegend=False
)

fig.show()

print(f"\nIncident counts by season:")
for season, count in season_counts.items():
    print(f"{season}: {count:,}")


Incident counts by season:
Winter: 60,038
Spring: 61,198
Summer: 60,921
Fall: 57,520


In [41]:
# Define US regions
state_to_region = {
    # Northeast
    'Connecticut': 'Northeast', 'Maine': 'Northeast', 'Massachusetts': 'Northeast',
    'New Hampshire': 'Northeast', 'Rhode Island': 'Northeast', 'Vermont': 'Northeast',
    'New Jersey': 'Northeast', 'New York': 'Northeast', 'Pennsylvania': 'Northeast',
    
    # Midwest
    'Illinois': 'Midwest', 'Indiana': 'Midwest', 'Michigan': 'Midwest', 'Ohio': 'Midwest',
    'Wisconsin': 'Midwest', 'Iowa': 'Midwest', 'Kansas': 'Midwest', 'Minnesota': 'Midwest',
    'Missouri': 'Midwest', 'Nebraska': 'Midwest', 'North Dakota': 'Midwest', 'South Dakota': 'Midwest',
    
    # South
    'Delaware': 'South', 'Florida': 'South', 'Georgia': 'South', 'Maryland': 'South',
    'North Carolina': 'South', 'South Carolina': 'South', 'Virginia': 'South',
    'District of Columbia': 'South', 'West Virginia': 'South', 'Alabama': 'South',
    'Kentucky': 'South', 'Mississippi': 'South', 'Tennessee': 'South', 'Arkansas': 'South',
    'Louisiana': 'South', 'Oklahoma': 'South', 'Texas': 'South',
    
    # West
    'Arizona': 'West', 'Colorado': 'West', 'Idaho': 'West', 'Montana': 'West',
    'Nevada': 'West', 'New Mexico': 'West', 'Utah': 'West', 'Wyoming': 'West',
    'Alaska': 'West', 'California': 'West', 'Hawaii': 'West', 'Oregon': 'West',
    'Washington': 'West'
}

# Add region column to dataframe
df['region'] = df['state'].map(state_to_region)

# Count incidents by region
region_counts = df['region'].value_counts().reset_index()
region_counts.columns = ['region', 'count']

print("Incident counts by region:")
for idx, row in region_counts.iterrows():
    print(f"{row['region']}: {row['count']:,}")

# For the map, we need to show regions by coloring states
# Create a dataframe with state codes and their region counts
df['state_code'] = df['state'].map(state_abbrev)

# Get total region count for each state
state_region_counts = df.groupby(['state_code', 'region']).size().reset_index(name='count')

# Create choropleth map colored by region
# We'll use the region as the color, but show incident count in hover
region_colors = {'Northeast': 1, 'Midwest': 2, 'South': 3, 'West': 4}
state_region_counts['region_num'] = state_region_counts['region'].map(region_colors)

fig = go.Figure(data=go.Choropleth(
    locations=state_region_counts['state_code'],
    z=state_region_counts['region_num'],
    locationmode='USA-states',
    colorscale=[[0, 'lightblue'], [0.33, 'lightgreen'], [0.66, 'lightyellow'], [1, 'lightcoral']],
    showscale=False,
    text=state_region_counts['region'],
    customdata=state_region_counts['count'],
    hovertemplate='<b>%{location}</b><br>Region: %{text}<br>Incidents: %{customdata:,}<extra></extra>'
))

fig.update_layout(
    title_text='Gun Violence Incidents by US Region (2013-2018)',
    geo_scope='usa',
    height=600,
    width=1000,
    annotations=[
        dict(
            x=0.5, y=-0.1,
            xref='paper', yref='paper',
            text='Northeast (Blue) | Midwest (Green) | South (Yellow) | West (Red)',
            showarrow=False,
            font=dict(size=12)
        )
    ]
)

fig.show()

print(f"\nTotal incidents: {region_counts['count'].sum():,}")

Incident counts by region:
South: 106,664
Midwest: 61,044
Northeast: 36,314
West: 35,655



Total incidents: 239,677


In [42]:
# Create bar chart for incidents by region
region_order = ['Northeast', 'Midwest', 'South', 'West']
region_counts_sorted = region_counts.set_index('region').reindex(region_order)

fig = go.Figure(data=[
    go.Bar(
        x=region_counts_sorted.index,
        y=region_counts_sorted['count'],
        marker_color=['lightblue', 'lightgreen', 'lightyellow', 'lightcoral'],
        text=region_counts_sorted['count'],
        textposition='auto',
        texttemplate='%{text:,}',
        hovertemplate='<b>%{x}</b><br>Incidents: %{y:,}<extra></extra>'
    )
])

fig.update_layout(
    title='Gun Violence Incidents by US Region (2013-2018)',
    xaxis_title='Region',
    yaxis_title='Number of Incidents',
    height=500,
    width=800,
    showlegend=False
)

fig.show()

In [43]:
# Get top 10 cities by incident count
top_10_cities = df['city_or_county'].value_counts().head(10).reset_index()
top_10_cities.columns = ['city', 'count']

print("Top 10 cities with gun violence incidents:")
for idx, row in top_10_cities.iterrows():
    print(f"{idx+1}. {row['city']}: {row['count']:,}")

# Create descending bar chart
fig = go.Figure(data=[
    go.Bar(
        x=top_10_cities['city'],
        y=top_10_cities['count'],
        marker_color='crimson',
        text=top_10_cities['count'],
        textposition='auto',
        texttemplate='%{text:,}',
        hovertemplate='<b>%{x}</b><br>Incidents: %{y:,}<extra></extra>'
    )
])

fig.update_layout(
    title='Top 10 Cities with Gun Violence Incidents (2013-2018)',
    xaxis_title='City',
    yaxis_title='Number of Incidents',
    height=500,
    width=1000,
    showlegend=False,
    xaxis={'categoryorder':'total descending'}
)

fig.show()

Top 10 cities with gun violence incidents:
1. Chicago: 10,814
2. Baltimore: 3,943
3. Washington: 3,279
4. New Orleans: 3,071
5. Philadelphia: 2,963
6. Saint Louis: 2,501
7. Houston: 2,501
8. Milwaukee: 2,487
9. Jacksonville: 2,448
10. Memphis: 2,386


In [44]:
# Get geographic data for top 10 cities
# For each city, get one representative lat/lon (using the first occurrence)
top_10_city_names = top_10_cities['city'].tolist()
city_geo_data = []

for city in top_10_city_names:
    city_data = df[df['city_or_county'] == city].iloc[0]
    city_geo_data.append({
        'city': city,
        'lat': city_data['latitude'],
        'lon': city_data['longitude'],
        'state': city_data['state'],
        'count': top_10_cities[top_10_cities['city'] == city]['count'].values[0]
    })

city_geo_df = pd.DataFrame(city_geo_data)

# Create scatter geo map
fig = go.Figure()

# Add base map of US states
fig.add_trace(go.Choropleth(
    locations=['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 
               'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
               'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
               'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
               'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'DC'],
    z=[0]*51,
    locationmode='USA-states',
    colorscale=[[0, 'lightgray'], [1, 'lightgray']],
    showscale=False,
    hoverinfo='skip'
))

# Add scatter points for top 10 cities
fig.add_trace(go.Scattergeo(
    lon=city_geo_df['lon'],
    lat=city_geo_df['lat'],
    text=city_geo_df['city'],
    mode='markers+text',
    marker=dict(
        size=city_geo_df['count'] / 100,  # Scale marker size by incident count
        color='crimson',
        line=dict(width=1, color='darkred'),
        sizemode='area'
    ),
    textposition='top center',
    textfont=dict(size=10, color='black'),
    customdata=city_geo_df[['state', 'count']],
    hovertemplate='<b>%{text}</b><br>State: %{customdata[0]}<br>Incidents: %{customdata[1]:,}<extra></extra>'
))

fig.update_layout(
    title='Top 10 Cities with Gun Violence Incidents (2013-2018)',
    geo=dict(
        scope='usa',
        projection_type='albers usa',
        showland=True,
        landcolor='rgb(243, 243, 243)',
        coastlinecolor='rgb(204, 204, 204)',
    ),
    height=600,
    width=1000,
    showlegend=False
)

fig.show()

print("\nTop 10 cities locations:")
for idx, row in city_geo_df.iterrows():
    print(f"{row['city']}, {row['state']}: {row['count']:,} incidents")


Top 10 cities locations:
Chicago, Illinois: 10,814 incidents
Baltimore, Maryland: 3,943 incidents
Washington, District of Columbia: 3,279 incidents
New Orleans, Louisiana: 3,071 incidents
Philadelphia, Pennsylvania: 2,963 incidents
Saint Louis, Missouri: 2,501 incidents
Houston, Texas: 2,501 incidents
Milwaukee, Wisconsin: 2,487 incidents
Jacksonville, Florida: 2,448 incidents
Memphis, Tennessee: 2,386 incidents


In [45]:
# display df["gun_type"] unique values
# print(df["gun_type"].unique())
# print(df["gun_type"].value_counts())
print(df["gun_type"].value_counts(normalize=True))
print(len(df["gun_type"].value_counts(normalize=True)))

gun_type
0::Unknown                                                                                                                                                                                                          0.667202
0::Handgun                                                                                                                                                                                                          0.092836
0::9mm                                                                                                                                                                                                              0.032797
0::Unknown||1::Unknown                                                                                                                                                                                              0.017187
0::22 LR                                                                                                   

In [46]:
# Create a flexible function to count any gun type
def count_gun_type(gun_type_str, target_gun_type):
    """
    Count occurrences of a specific gun type in the gun_type string.
    
    Parameters:
    - gun_type_str: The gun_type field from the dataset
    - target_gun_type: The specific gun type to count (e.g., "Handgun", "9mm", etc.)
    
    Returns:
    - Count of the specified gun type
    """
    if pd.isna(gun_type_str):
        return 0
    # Split by || to get individual guns
    guns = str(gun_type_str).split('||')
    # Count how many match the target gun type
    count = 0
    for gun in guns:
        if '::' in gun:
            gun_name = gun.split('::')[1].strip()
            if gun_name == target_gun_type:
                count += 1
    return count

# List of gun types to analyze
gun_types = [
    "Handgun",
    "9mm",
    "22 LR",
    "Unknown",
    "38 Spl",
    "223 Rem [AR-15]",
    "Shotgun",
    "Rifle",
    "45 Auto",
    "44 Mag",
    "357 Mag"
]

# Create columns for each gun type
for gun_type in gun_types:
    # Create column name by replacing spaces and brackets with underscores
    col_name = f"{gun_type} Count".replace(" ", "_").replace("[", "").replace("]", "").replace("-", "_")
    df[col_name] = df['gun_type'].apply(lambda x: count_gun_type(x, gun_type))

# Display comprehensive results
print("="*70)
print("GUN TYPE ANALYSIS - COMPREHENSIVE SUMMARY")
print("="*70)
print(f"\n{'Gun Type':<20} {'Total Guns':<15} {'Cases Involved':<15}")
print("-"*70)

for gun_type in gun_types:
    col_name = f"{gun_type} Count".replace(" ", "_").replace("[", "").replace("]", "").replace("-", "_")
    total_guns = df[col_name].sum()
    cases_involved = (df[col_name] > 0).sum()
    print(f"{gun_type:<20} {total_guns:>14,} {cases_involved:>14,}")

print("="*70)

GUN TYPE ANALYSIS - COMPREHENSIVE SUMMARY

Gun Type             Total Guns      Cases Involved 
----------------------------------------------------------------------
Handgun                      25,038         17,609
9mm                           6,448          6,040
22 LR                         3,346          3,093
Unknown                     130,838         98,892
38 Spl                        1,809          1,758
223 Rem [AR-15]               1,613          1,248
Shotgun                       4,263          3,560
Rifle                         5,268          3,196
45 Auto                       2,360          2,234
44 Mag                          197            192
357 Mag                         822            800


In [47]:
df.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,num_victims,num_suspects,num_females,num_males,num_adults,num_teens,num_adult_suspects,num_adult_victims,num_teen_suspects,num_teen_victims,num_male_suspects,num_male_victims,num_female_suspects,num_female_victims,n_guns_stolen,n_guns_unknown,n_guns_not_stolen,season,region,state_code,Handgun_Count,9mm_Count,22_LR_Count,Unknown_Count,38_Spl_Count,223_Rem_AR_15_Count,Shotgun_Count,Rifle_Count,45_Auto_Count,44_Mag_Count,357_Mag_Count
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN,4,1,1,3,5,0,1,4,0,0,0,3,1,0,0,0,0,Winter,Northeast,PA,0,0,0,0,0,0,0,0,0,0,0
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0,4,1,0,1,4,0,0,4,0,0,0,1,0,0,0,0,0,Winter,West,CA,0,0,0,0,0,0,0,0,0,0,0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,0::Unknown||1::Unknown,0::Unknown||1::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,Cotton Club,-82.1377,2.0,NaN,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0,3,2,0,5,5,0,2,3,0,0,2,3,0,0,0,2,0,Winter,Midwest,OH,0,0,0,2,0,0,0,0,0,0,0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Off...",39.6518,NaN,-104.8020,NaN,NaN,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0,3,1,1,3,4,0,1,3,0,0,1,2,0,1,0,0,0,Winter,West,CO,0,0,0,0,0,0,0,0,0,0,0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,0::Unknown||1::Unknown,0::Handgun||1::Handgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3

In [48]:
# Examine participant_gender patterns
print("Sample participant_gender values:")
print(df["participant_gender"].head(20))
print("\n" + "="*70)

# Check unique gender values
sample_entries = []
for entry in df["participant_gender"].dropna().head(100):
    genders = str(entry).split('||')
    for gender in genders:
        if '::' in gender:
            gender_value = gender.split('::')[1].strip()
            if gender_value not in sample_entries:
                sample_entries.append(gender_value)

print(f"\nUnique gender values found: {sample_entries}")
print("="*70)

Sample participant_gender values:
0                  0::Male||1::Male||3::Male||4::Female
1                                               0::Male
2           0::Male||1::Male||2::Male||3::Male||4::Male
3                  0::Female||1::Male||2::Male||3::Male
4                0::Female||1::Male||2::Male||3::Female
5     0::Female||1::Female||2::Female||3::Female||4:...
6     0::Male||1::Female||2::Male||3::Female||4::Fem...
7           0::Male||1::Male||2::Male||3::Male||4::Male
8           0::Male||1::Male||2::Male||3::Male||4::Male
9                                               0::Male
10                   0::Male||1::Male||2::Male||3::Male
11                                              0::Male
12          0::Male||1::Male||2::Male||3::Male||4::Male
13    0::Female||1::Female||2::Male||3::Male||4::Mal...
14          0::Male||1::Male||2::Male||3::Male||4::Male
15    0::Female||1::Female||2::Female||3::Female||4:...
16    0::Male||1::Male||2::Female||4::Male||5::Male|...
17            

In [49]:
# Fix num_males and num_females using participant_gender

# Store old values for comparison
old_num_males = df['num_males'].copy()
old_num_females = df['num_females'].copy()

def count_gender(participant_gender_str, target_gender):
    """
    Count occurrences of a specific gender in participant_gender string.
    
    Parameters:
    - participant_gender_str: The participant_gender field from the dataset
    - target_gender: "Male" or "Female"
    
    Returns:
    - Count of the specified gender
    """
    if pd.isna(participant_gender_str):
        return 0
    # Split by || to get individual participants
    participants = str(participant_gender_str).split('||')
    # Count how many match the target gender
    count = 0
    for participant in participants:
        if '::' in participant:
            gender = participant.split('::')[1].strip()
            if gender == target_gender:
                count += 1
    return count

# Replace num_males and num_females with corrected counts
df['num_males'] = df['participant_gender'].apply(lambda x: count_gender(x, 'Male'))
df['num_females'] = df['participant_gender'].apply(lambda x: count_gender(x, 'Female'))

# Display comparison
print("="*70)
print("GENDER COUNT CORRECTION")
print("="*70)

print("\nBefore correction:")
print(f"Total males (old): {old_num_males.sum():,}")
print(f"Total females (old): {old_num_females.sum():,}")

print("\nAfter correction:")
print(f"Total males (new): {df['num_males'].sum():,}")
print(f"Total females (new): {df['num_females'].sum():,}")

print("\nSample comparison (first 20 rows):")
comparison_df = pd.DataFrame({
    'participant_gender': df['participant_gender'].head(20),
    'old_males': old_num_males.head(20),
    'new_males': df['num_males'].head(20),
    'old_females': old_num_females.head(20),
    'new_females': df['num_females'].head(20)
})
print(comparison_df.to_string())

print("\n" + "="*70)
print("✓ num_males and num_females have been corrected!")
print("="*70)

GENDER COUNT CORRECTION

Before correction:
Total males (old): 304,102
Total females (old): 42,376

After correction:
Total males (new): 304,102
Total females (new): 42,376

Sample comparison (first 20 rows):
                                                   participant_gender  old_males  new_males  old_females  new_females
0                                0::Male||1::Male||3::Male||4::Female          3          3            1            1
1                                                             0::Male          1          1            0            0
2                         0::Male||1::Male||2::Male||3::Male||4::Male          5          5            0            0
3                                0::Female||1::Male||2::Male||3::Male          3          3            1            1
4                              0::Female||1::Male||2::Male||3::Female          2          2            2            2
5        0::Female||1::Female||2::Female||3::Female||4::Male||5::Male          2   